In [1]:
cd /home/muhammadbsheikh/workspace/try

/home/muhammadbsheikh/workspace/try


In [2]:
def extract_features(model,dl):    
    model.eval()
    device = 'cuda:0'
    model.cuda(device)
    with torch.no_grad():
        features = []
        for images,labels in tqdm(dl, disable=True):       
            
            images = images.to(device)
            #labels = labels.to(device)

            output = model(images)
            
            #print(labels)

            feat = output.data.cpu().numpy().copy(),labels
            print(output.size(),labels)
            features.append(feat)     
            

    return features


def flatten_list(t):
    flat_list = [item for sublist in t for item in sublist]
    flat_list = np.array(flat_list)
    return flat_list

In [3]:
rep = '8'
# import standard PyTorch modules
import os
os.environ['CUDA_VISIBLE_DEVICES']="0,1,2,3"
import torch
import torch.nn as nn
import torch.nn.functional as F
import pretrainedmodels
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter # TensorBoard support

# import torchvision module to handle image manipulation
import torchvision
from torchvision import transforms
from torch.autograd import Variable

# calculate train time, writing train data to files etc.
import time
import numpy as np
from tqdm import tqdm
from IPython.display import clear_output

from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

import scipy
import scipy.fft

print(torch.__version__)
print(torchvision.__version__)

torch.set_printoptions(linewidth=120)
torch.set_grad_enabled(True)     # On by default, leave it here for clarity


#set device
if torch.cuda.is_available():
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print(torch.cuda.get_device_properties(device),torch.cuda.set_device(device),torch.cuda.current_device())

#TRAIN_DATA_PATH = "./datax/"+rep+"/train/"
#TEST_DATA_PATH = "./datax/"+rep+"/test/"
TRAIN_DATA_PATH = "/mnt/bdata/Data/bilal_data/ksounds_chrom/train"
TEST_DATA_PATH = "/mnt/bdata/Data/bilal_data/ksounds_chrom/val"

transform = transforms.Compose([    
    transforms.Resize(299), # preffered size for network
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225] )
    ])

train_data = torchvision.datasets.ImageFolder(root=TRAIN_DATA_PATH, transform=transform)
test_data = torchvision.datasets.ImageFolder(root=TEST_DATA_PATH, transform=transform)

train_data_loader = torch.utils.data.DataLoader(train_data, batch_size=1, shuffle=False,  num_workers=4)
test_data_loader = torch.utils.data.DataLoader(test_data, batch_size=1, shuffle=False,  num_workers=4)

#prepare model
model_name = 'inceptionresnetv2' # could be fbresnet152 or inceptionresnetv2
model = pretrainedmodels.__dict__[model_name](num_classes=1000, pretrained='imagenet')

model.last_linear = nn.Identity() #freeze the model
#num_ftrs = model.last_linear.in_features
#model.last_linear = nn.Linear(num_ftrs, 50)

for p in model.parameters():
    p.requires_grad =False
    
PATH = "models/IRv2.pt"

torch.save(model,PATH)
#model = torch.load(PATH)

optimizer = optim.Adam(model.parameters(), lr=0.0005)

def train(model,loader,epochs=60):
    
    model.train()   
    print('Training...')
    for epoch in range(epochs):
        start = time.time()
        model.train()
        
        running_loss=0

        for i,batch in enumerate(loader,0):
                images = batch[0]
                labels = batch[1]
                images = images.to(device)
                labels = labels.to(device)
                model.to(device)
                
                preds = model(images)
                loss = F.cross_entropy(preds, labels) # Adam, SGD, RSPROP

                optimizer.zero_grad()
                loss = Variable(loss, requires_grad = True)
                loss.backward()
                optimizer.step()

                #running_loss+=loss.data
                running_loss += loss.item()*images.size(0)

                if i%9==0:
                    end=time.time()
                    #print ('[epoch %d,imgs %5d] time: %0.3f s'%(epoch+1,(i+1)*4,(end-start)))
                    #print ('[epoch %d,imgs %5d] loss: %.7f  time: %0.3f s'%(epoch+1,(i+1)*128,running_loss/100,(end-start)))
                    #tb.add_scalar('Loss', loss, epoch+1)
                    start=time.time()
        epoch_loss = running_loss / len(loader.dataset)
        print(f'[epoch %d/%d] loss: %.3f '%(epoch,epochs,epoch_loss))
    return model 
    

#model = train(model,train_data_loader)



1.8.1
0.9.1
_CudaDeviceProperties(name='GeForce GTX 1080 Ti', major=6, minor=1, total_memory=11178MB, multi_processor_count=28) None 0


In [4]:
len(train_data_loader.dataset)

19349

In [ ]:
train_features = extract_features(model,train_data_loader)
test_features = extract_features(model,test_data_loader)

In [6]:
len(train_features),len(test_features)

(19349, 1344)

In [ ]:
count =0
for x in train_data_loader.dataset.samples:
    n = x[0].split('/')[4]+'/v_'+x[0].split('/')[5].split('.')[0]
    with open('/home/muhammadbsheikh/workspace/projects/githubs_repos/tsn/tsn-pytorch/ucf_aud2vid_trainfiles1_new.txt') as f:
        if n in f.read():
            count=count+1
        else:
            print(n)
print(count)
count =0
for x in test_data_loader.dataset.samples:
    n = x[0].split('/')[4]+'/v_'+x[0].split('/')[5].split('.')[0]
    with open('/home/muhammadbsheikh/workspace/projects/githubs_repos/tsn/tsn-pytorch/ucf_aud2vid_testfiles1_new.txt') as f:
        if n in f.read():
            count=count+1
        else:
            print(n)
print(count)

In [ ]:
#train
audio_train_features = [x[0] for x in train_features]
audio_train_labels = [x[1] for x in train_features]
np.save('/home/muhammadbsheikh/workspace/projects/githubs_repos/tsn/tsn-pytorch2/tsn-pytorch/features/audio_features-train'+rep+'.npy',np.array(audio_train_features))
np.save('/home/muhammadbsheikh/workspace/projects/githubs_repos/tsn/tsn-pytorch2/tsn-pytorch/features/audio_labels-train'+rep+'.npy',np.array(audio_train_labels))
#test
audio_test_features = [x[0] for x in test_features]
audio_test_labels = [x[1] for x in test_features]
np.save('/home/muhammadbsheikh/workspace/projects/githubs_repos/tsn/tsn-pytorch2/tsn-pytorch/features/audio_features-test'+rep+'.npy',np.array(audio_test_features))
np.save('/home/muhammadbsheikh/workspace/projects/githubs_repos/tsn/tsn-pytorch2/tsn-pytorch/features/audio_labels-test'+rep+'.npy',np.array(audio_test_labels))

#### Ksounds Feature Save

In [ ]:
rep = 'ksounds'
#train
audio_train_features = [x[0] for x in train_features]
audio_train_labels = [x[1] for x in train_features]
np.save('/home/muhammadbsheikh/workspace/projects/ksounds_MAiVAR/features/audio_features-train'+rep+'.npy',np.array(audio_train_features))
np.save('/home/muhammadbsheikh/workspace/projects/ksounds_MAiVAR/features/audio_labels-train'+rep+'.npy',np.array(audio_train_labels))
#test
audio_test_features = [x[0] for x in test_features]
audio_test_labels = [x[1] for x in test_features]
np.save('/home/muhammadbsheikh/workspace/projects/ksounds_MAiVAR/features/audio_features-test'+rep+'.npy',np.array(audio_test_features))
np.save('/home/muhammadbsheikh/workspace/projects/ksounds_MAiVAR/features/audio_labels-test'+rep+'.npy',np.array(audio_test_labels))

In [ ]:
#feat,lbls = extract_features(model,train_data_loader)
test_feat,test_lbls = extract_features(model,test_data_loader)
lbls = flatten_list(lbls)
test_lbls  =flatten_list(test_lbls) # flatting the lbls

In [ ]:
feat.shape,len(lbls),test_feat.shape,len(test_lbls)

In [ ]:
from sklearn import metrics
print('Train-Without FFT')
#SVM
svm_preds = SVC(kernel='linear').fit(feat,lbls).predict(test_feat)
print("SVM Accuracy:",metrics.accuracy_score(test_lbls, svm_preds))

In [ ]:
i = 0 #video index 4797
j = 0 # neuron index 1000

def hstft(S1):
    S21, S22 = np.split(S1,2)  # 500+500
    S311, S312 = np.split(S21,2) # 500    
    S321, S322 = np.split(S22,2) # 500
    #S1.shape,S21.shape,S22.shape,S311.shape,S312.shape,S321.shape,S322.shape    
    return np.concatenate((S1,S21,S22,S311,S312,S321,S322))

def alpha(features):
    A = np.zeros((len(features),4608))
    for i,f in enumerate(features):
        h = scipy.fft.fft(f)    
        #print(f.shape,h.shape)
        #print(h.shape)
        h = hstft(h)
        #print(h.shape)
        A[i,:] = h.real
    return A

train_alpha = alpha(feat)
test_alpha = alpha(test_feat)
train_alpha.shape,test_alpha.shape

In [ ]:
X = train_alpha
Y = lbls
test_x = test_alpha
test_y = test_lbls
X.shape,Y.shape,test_x.shape,test_y.shape

In [ ]:
from sklearn.ensemble import RandomForestClassifier 
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics

#SVM
clf = SVC(kernel='linear').fit(X,Y)
preds = clf.predict(test_x)
print("SVM Accuracy:",metrics.accuracy_score(test_y, preds))

#KNN

knn_clf = KNeighborsClassifier(n_neighbors=3).fit(X,Y)
knn_preds = knn_clf.predict(test_x)
print("KNN Accuracy:",metrics.accuracy_score(test_y, knn_preds))

#Random Forest
rf_clf = RandomForestClassifier(n_estimators=100).fit(X,Y)
rf_preds = rf_clf.predict(test_x)
print("RF Accuracy:",metrics.accuracy_score(test_y, rf_preds))